In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
combined = pd.read_csv("../input/g-research-crypto-forecasting/train.csv")

In [ ]:
asset_details = pd.read_csv("../input/g-research-crypto-forecasting/asset_details.csv")
#asset id maps to the training set with the time series data.  The weight of the cryptocurrency is also listed
asset_details.index = asset_details["Asset_ID"]
combined['datetime'] = pd.to_datetime(combined['timestamp'], unit='s')
combined["Asset_Name"] = asset_details.loc[combined.loc[:,"Asset_ID"],"Asset_Name"].to_numpy()
#combined.loc[combined["Target"].isna(),"Target"]=combined["Target"].mean(skipna=True)

In [ ]:
def calculate_target(data: pd.DataFrame, details: pd.DataFrame, price_column: str):
    ids = list(details.Asset_ID)
    asset_names = list(details.Asset_Name)
    weights = np.array(list(details.Weight))

    all_timestamps = np.sort(data['timestamp'].unique())
    targets = pd.DataFrame(index=all_timestamps)

    for i, id in enumerate(ids):
        asset = data[data.Asset_ID == id].set_index(keys='timestamp')
        price = pd.Series(index=all_timestamps, data=asset[price_column])
        targets[asset_names[i]] = (
            price.shift(periods=-16) /
            price.shift(periods=-1)
        ) - 1
    
    targets['marketReturn'] = np.average(targets.fillna(0), axis=1, weights=weights)
    
    m = targets['marketReturn']

    num = targets.multiply(m.values, axis=0).rolling(3734).mean().values
    denom = m.multiply(m.values, axis=0).rolling(3734).mean().values
    beta = np.nan_to_num(num.T / denom, nan=0., posinf=0., neginf=0.)

    for i, id in enumerate(ids):
        targets["BetaRolling"+asset_names[i]] = beta[i,:]



    targets['datetime'] = pd.to_datetime(targets.index, unit='s')
    targets["year"] = pd.DatetimeIndex(targets['datetime']).year
    targets["month"] = pd.DatetimeIndex(targets['datetime']).month


    return targets

targets = (calculate_target(combined,asset_details,"Close"))

In [ ]:
def createDatasetCoin(data,suffix):
    data["Target"+suffix] = data["Target"]
    return data[["Target"+suffix,"datetime"]]

def createDataset(targets):
    for coin in asset_details.Asset_Name:
        data = combined.loc[combined["Asset_Name"]==coin,:].reset_index(drop=True) 
        data = createDatasetCoin(data, coin)
        targets=pd.merge(targets,data, how="left",on="datetime")

    return targets

targets = createDataset(targets)

In [ ]:
targets=targets.replace([np.inf,-np.inf],np.nan)
targets = targets.fillna(method="ffill")
targets = targets.fillna(method="bfill")
targets = targets[3750:-16]

In [ ]:
train = targets.loc[(targets["datetime"]>='2020-06-13 00:00:00') | (targets["datetime"]<'2021-06-13 00:00:00'),:]
test = targets.loc[targets["datetime"]>='2021-06-13 00:00:00',:]
features = ["Bitcoin","BetaRollingBitcoin"] 

In [ ]:
#lgbm model to predict a regression var
import lightgbm as lgb
targetVar = "Target"+"Bitcoin"
train_data = lgb.Dataset(train.loc[:,features], label=train.loc[:,targetVar])
test_data= lgb.Dataset(test.loc[:,features], label=test.loc[:,targetVar])
test_set=test.loc[:,features]
param = {'num_leaves': 64, 'objective': 'regression', 'learning_rate':.01, 'max_depth':16, 'force_col_wise':True}
param['metric'] = ['l1']

num_round = 500
lightModel = lgb.train(param, train_data, num_round, valid_sets=[train_data,test_data], early_stopping_rounds=30)
predsReg=lightModel.predict(test_set)

In [ ]:
np.corrcoef(test[targetVar],predsReg)

In [ ]:
train = targets.loc[(targets["datetime"]>='2020-06-13 00:00:00') | (targets["datetime"]<'2021-06-13 00:00:00'),:]
test = targets.loc[targets["datetime"]>='2021-06-13 00:00:00',:]
features = ['Bitcoin Cash', 'Binance Coin', 'Bitcoin', 'EOS.IO', 'Ethereum Classic',
       'Ethereum', 'Litecoin', 'Monero', 'TRON', 'Stellar', 'Cardano', 'IOTA',
       'Maker', 'Dogecoin',"BetaRollingBitcoin"] 

In [ ]:
#lgbm model to predict a regression var
import lightgbm as lgb
targetVar = "Target"+"Bitcoin"
train_data = lgb.Dataset(train.loc[:,features], label=train.loc[:,targetVar])
test_data= lgb.Dataset(test.loc[:,features], label=test.loc[:,targetVar])
test_set=test.loc[:,features]
param = {'num_leaves': 64, 'objective': 'regression', 'learning_rate':.01, 'max_depth':16, 'force_col_wise':True}
param['metric'] = ['l1']

num_round = 500
lightModel = lgb.train(param, train_data, num_round, valid_sets=[train_data,test_data], early_stopping_rounds=30)
predsReg=lightModel.predict(test_set)

In [ ]:
np.corrcoef(test[targetVar],predsReg)